In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import re
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time

# Atur agar semua kolom terlihat
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)
pd.options.display.max_rows = None
pd.options.display.max_columns = None 

# df = pd.read_csv('AB_US_2023.csv')
# dt = pd.read_csv('AB_US_2020.csv')

In [17]:
df = pd.read_csv("airbnb-listings-us-cleaned-o.csv", low_memory=False)

In [18]:
df.to_csv("airbnb-nyoba.csv", index=False)

In [19]:
df = pd.read_csv("airbnb-nyoba.csv", low_memory=False)

In [10]:
df.isnull().sum() / len(df) * 100

Last Scraped                      0.0
Experiences Offered               0.0
Host Since                        0.0
Host Response Time                0.0
Host Response Rate                0.0
Host Neighbourhood                0.0
Host Total Listings Count         0.0
Host Verifications                0.0
Street                            0.0
Neighbourhood                     0.0
City                              0.0
State                             0.0
Zipcode                           0.0
Latitude                          0.0
Longitude                         0.0
Property Type                     0.0
Room Type                         0.0
Accommodates                      0.0
Bathrooms                         0.0
Bedrooms                          0.0
Beds                              0.0
Bed Type                          0.0
Amenities                         0.0
Price                             0.0
Cleaning Fee                      0.0
Guests Included                   0.0
Extra People

Karena sebelumnya amenities sudah dipisah maka di drop

In [36]:
df.drop('Amenities', axis=1, inplace=True)

In [20]:
df['Last Scraped'].nunique()

25

In [15]:
df['Last Scraped'].value_counts()

Last Scraped
2017-05-03    45483
2017-05-04    17564
2017-03-07     9234
2017-04-02     8506
2017-05-10     6881
2016-07-07     6156
2017-05-11     5764
2017-06-02     5259
2017-05-05     4904
2016-01-04     3724
2017-04-07     3540
2016-09-07     3485
2017-05-02     3315
2016-09-06     3179
2016-05-04     1642
2016-05-21      717
2016-05-19      494
2016-05-18      464
2016-05-20      398
2016-05-17      353
2015-10-18      336
2015-10-17      302
2015-10-16      130
2015-10-19       24
2017-04-08        2
Name: count, dtype: int64

In [21]:
df['Experiences Offered'].unique()

array(['none'], dtype=object)

In [22]:
df.drop('Experiences Offered', axis=1, inplace=True)

Encoding feature ordinal

**Host Response Time**

In [23]:
df['Host Response Time'].unique()

array(['within an hour', 'within a day', 'within a few hours',
       'a few days or more'], dtype=object)

In [24]:
df.loc[df['Host Response Time'] == "within an hour", "Host Response Time"] = 0
df.loc[df['Host Response Time'] == "within a few hours", "Host Response Time"] = 1
df.loc[df['Host Response Time'] == "within a day", "Host Response Time"] = 2
df.loc[df['Host Response Time'] == "a few days or more", "Host Response Time"] = 3

In [25]:
df['Host Response Time'].unique()

array([0, 2, 1, 3], dtype=object)

Encoding features One-Hot

**Room Type**

In [29]:
df = pd.get_dummies(df, columns=['Room Type'], drop_first=True)

**Bed Type**

In [32]:
df = pd.get_dummies(df, columns=['Bed Type'], drop_first=True)

**Cancellation Policy**

In [33]:
df = pd.get_dummies(df, columns=['Cancellation Policy'], drop_first=True)

In [37]:
df.head()

,Last Scraped,Host Since,Host Response Time,Host Response Rate,Host Neighbourhood,Host Total Listings Count,Host Verifications,Street,Neighbourhood,City,State,Zipcode,Latitude,Longitude,Property Type,Accommodates,Bathrooms,Bedrooms,Beds,Price,Cleaning Fee,Guests Included,Extra People,Minimum Nights,Maximum Nights,Calendar Updated,Availability 30,Availability 60,Availability 90,Availability 365,Calendar last Scraped,Number of Reviews,Review Scores Rating,Review Scores Accuracy,Review Scores Cleanliness,Review Scores Checkin,Review Scores Communication,Review Scores Location,Review Scores Value,Calculated host listings count,Reviews per Month,Wireless Internet,Heating,Kitchen,Essentials,TV,Smoke detector,Shampoo,Internet,Washer,Dryer,Room Type_Private room,Room Type_Shared room,Bed Type_Couch,Bed Type_Futon,Bed Type_Pull-out Sofa,Bed Type_Real Bed,Cancellation Policy_long_term,Cancellation Policy_moderate,Cancellation Policy_no_refunds,Cancellation Policy_strict,Cancellation Policy_super_strict_30,Cancellation Policy_super_strict_60
0,2017-05-02,2014-11-22 00:00:00,0,100.0,Woodland Hills/Warner Center,1,"email,phone,jumio","Woodland Hills/Warner Center, Los Angeles, CA 91364, United States",Woodland Hills,Los Angeles,CA,91364,34.147756,-118.591338,Apartment,2.0,1,1,1,150.0,25.0,1.0,0.0,1.0,1125.0,29 months ago,30.0,60.0,90.0,365.0,2017-05-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.00,True,True,False,True,True,True,True,True,False,False,True,False,False,False,False,True,False,False,False,False,False,False
1,2017-05-03,2016-02-02 00:00:00,2,100.0,Woodland Hills/Warner Center,1,"email,phone,facebook,reviews,kba","Woodland Hills/Warner Center, Los Angeles, CA 91364, United States",Woodland Hills,Los Angeles,CA,91364,34.164597,-118.600512,House,8.0,3,4,4,175.0,150.0,8.0,25.0,2.0,1125.0,today,9.0,19.0,28.0,28.0,2017-05-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.00,True,True,True,True,True,True,True,True,True,True,False,False,False,False,False,True,False,False,False,False,False,False
2,2017-05-03,2016-11-22 00:00:00,1,100.0,Woodland Hills/Warner Center,1,phone,"Woodland Hills/Warner Center, Los Angeles, CA 91367, United States",Woodland Hills,Los Angeles,CA,91367,34.183150,-118.645064,Guesthouse,3.0,1,1,1,79.0,55.0,1.0,25.0,2.0,26.0,yesterday,30.0,60.0,90.0,90.0,2017-05-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.00,True,True,True,True,True,True,False,True,True,True,False,False,False,False,False,True,False,False,False,False,False,False
3,2017-05-03,2011-04-13 00:00:00,1,100.0,Woodland Hills/Warner Center,2,"email,phone,reviews,kba","Woodland Hills/Warner Center, Woodland Hills, CA 91364, United States",Woodland Hills,Woodland Hills,CA,91364,34.148594,-118.596684,Other,2.0,1,1,1,175.0,100.0,1.0,0.0,2.0,5.0,2 weeks ago,12.0,42.0,72.0,72.0,2017-05-03,2.0,100.0,10.0,10.0,10.0,10.0,10.0,10.0,2.0,0.16,True,True,True,True,True,True,True,True,True,True,False,False,False,False,False,True,False,False,False,True,False,False
4,2017-05-03,2015-01-03 00:00:00,0,100.0,Studio City,36,"email,phone,facebook,reviews,jumio","Woodland Hills/Warner Center, Los Angeles, CA 91367, United States",Woodland Hills,Los Angeles,CA,91367,34.174245,-118.605876,Townhouse,6.0,2,3,4,450.0,200.0,6.0,50.0,3.0,1125.0,today,26.0,56.0,86.0,361.0,2017-05-03,8.0,95.0,9.0,10.0,10.0,10.0,9.0,10.0,29.0,0.79,True,True,True,True,True,True,False,True,True,True,False,False,False,False,False,True,False,True,False,False,False,False


nge convert kolom boolean menjadi 1/0

In [41]:
bool_cols = ['Wireless Internet', 'Heating', 'Kitchen', 'Essentials', 'TV', 
             'Smoke detector', 'Shampoo', 'Internet', 'Washer', 'Dryer',
             'Room Type_Private room', 'Room Type_Shared room', 'Bed Type_Couch',
             'Bed Type_Futon', 'Bed Type_Pull-out Sofa', 'Bed Type_Real Bed',
             'Cancellation Policy_long_term', 'Cancellation Policy_moderate',
             'Cancellation Policy_no_refunds', 'Cancellation Policy_strict',
             'Cancellation Policy_super_strict_30', 'Cancellation Policy_super_strict_60']
df[bool_cols] = df[bool_cols].astype(int)

In [42]:
df.head()

,Last Scraped,Host Since,Host Response Time,Host Response Rate,Host Neighbourhood,Host Total Listings Count,Host Verifications,Street,Neighbourhood,City,State,Zipcode,Latitude,Longitude,Property Type,Accommodates,Bathrooms,Bedrooms,Beds,Price,Cleaning Fee,Guests Included,Extra People,Minimum Nights,Maximum Nights,Calendar Updated,Availability 30,Availability 60,Availability 90,Availability 365,Calendar last Scraped,Number of Reviews,Review Scores Rating,Review Scores Accuracy,Review Scores Cleanliness,Review Scores Checkin,Review Scores Communication,Review Scores Location,Review Scores Value,Calculated host listings count,Reviews per Month,Wireless Internet,Heating,Kitchen,Essentials,TV,Smoke detector,Shampoo,Internet,Washer,Dryer,Room Type_Private room,Room Type_Shared room,Bed Type_Couch,Bed Type_Futon,Bed Type_Pull-out Sofa,Bed Type_Real Bed,Cancellation Policy_long_term,Cancellation Policy_moderate,Cancellation Policy_no_refunds,Cancellation Policy_strict,Cancellation Policy_super_strict_30,Cancellation Policy_super_strict_60
0,2017-05-02,2014-11-22 00:00:00,0,100.0,Woodland Hills/Warner Center,1,"email,phone,jumio","Woodland Hills/Warner Center, Los Angeles, CA 91364, United States",Woodland Hills,Los Angeles,CA,91364,34.147756,-118.591338,Apartment,2.0,1,1,1,150.0,25.0,1.0,0.0,1.0,1125.0,29 months ago,30.0,60.0,90.0,365.0,2017-05-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.00,1,1,0,1,1,1,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0
1,2017-05-03,2016-02-02 00:00:00,2,100.0,Woodland Hills/Warner Center,1,"email,phone,facebook,reviews,kba","Woodland Hills/Warner Center, Los Angeles, CA 91364, United States",Woodland Hills,Los Angeles,CA,91364,34.164597,-118.600512,House,8.0,3,4,4,175.0,150.0,8.0,25.0,2.0,1125.0,today,9.0,19.0,28.0,28.0,2017-05-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.00,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0
2,2017-05-03,2016-11-22 00:00:00,1,100.0,Woodland Hills/Warner Center,1,phone,"Woodland Hills/Warner Center, Los Angeles, CA 91367, United States",Woodland Hills,Los Angeles,CA,91367,34.183150,-118.645064,Guesthouse,3.0,1,1,1,79.0,55.0,1.0,25.0,2.0,26.0,yesterday,30.0,60.0,90.0,90.0,2017-05-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.00,1,1,1,1,1,1,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0
3,2017-05-03,2011-04-13 00:00:00,1,100.0,Woodland Hills/Warner Center,2,"email,phone,reviews,kba","Woodland Hills/Warner Center, Woodland Hills, CA 91364, United States",Woodland Hills,Woodland Hills,CA,91364,34.148594,-118.596684,Other,2.0,1,1,1,175.0,100.0,1.0,0.0,2.0,5.0,2 weeks ago,12.0,42.0,72.0,72.0,2017-05-03,2.0,100.0,10.0,10.0,10.0,10.0,10.0,10.0,2.0,0.16,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,1,0,0,0,1,0,0
4,2017-05-03,2015-01-03 00:00:00,0,100.0,Studio City,36,"email,phone,facebook,reviews,jumio","Woodland Hills/Warner Center, Los Angeles, CA 91367, United States",Woodland Hills,Los Angeles,CA,91367,34.174245,-118.605876,Townhouse,6.0,2,3,4,450.0,200.0,6.0,50.0,3.0,1125.0,today,26.0,56.0,86.0,361.0,2017-05-03,8.0,95.0,9.0,10.0,10.0,10.0,9.0,10.0,29.0,0.79,1,1,1,1,1,1,0,1,1,1,0,0,0,0,0,1,0,1,0,0,0,0


In [43]:
df.isnull().sum() / len(df) * 100

Last Scraped                           0.0
Host Since                             0.0
Host Response Time                     0.0
Host Response Rate                     0.0
Host Neighbourhood                     0.0
Host Total Listings Count              0.0
Host Verifications                     0.0
Street                                 0.0
Neighbourhood                          0.0
City                                   0.0
State                                  0.0
Zipcode                                0.0
Latitude                               0.0
Longitude                              0.0
Property Type                          0.0
Accommodates                           0.0
Bathrooms                              0.0
Bedrooms                               0.0
Beds                                   0.0
Price                                  0.0
Cleaning Fee                           0.0
Guests Included                        0.0
Extra People                           0.0
Minimum Nig